In [ ]:

import pandas as pd
import numpy as np
from scipy.stats import burr12

# parameters
file_path = "inputs.xlsx" 
sheet_name = "Sheet1"    
num_simulations = 10000   

# Burr12 parameters
burr12_params = {
    0: {'c': 2.0, 'd': 3.5, 'scale': 1.0},
    1: {'c': 1.5, 'd': 4.0, 'scale': 1.2},
    2: {'c': 2.2, 'd': 2.8, 'scale': 0.9},
    3: {'c': 1.8, 'd': 3.2, 'scale': 1.1},
    4: {'c': 2.5, 'd': 3.0, 'scale': 1.3},
    5: {'c': 2.1, 'd': 3.7, 'scale': 0.95}
}


df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=[0, 1, 2], header=None)
df.columns = ['Product Index', 'Nominal Demand', 'Stochastic Group']


df = df.reset_index(drop=True)      
df.index = df['Product Index']        


assert (df.index == range(1, len(df)+1)).all()

sim_data = {}

for product in df.index:
    demand = df.loc[product, 'Nominal Demand']
    group = df.loc[product, 'Stochastic Group']
    params = burr12_params[group]
    
    random_values = burr12.rvs(**params, size=num_simulations)
    simulated = demand * random_values
    sim_data[product] = simulated

sim_df = pd.DataFrame(sim_data)


sim_df = sim_df.T


sim_df.index.name = 'Product Index'
sim_df.columns = range(1, num_simulations + 1)  

sim_df.to_excel("demand_sumulation_output.xlsx", index=True)



In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

# number of k
NUM_CLUSTERS = 100


excel_path = 'demand_sumulation_output.xlsx'
df = pd.read_excel(excel_path, index_col=0)

scenario_matrix = df.T


kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
scenario_matrix['Cluster'] = kmeans.fit_predict(scenario_matrix)


cluster_means = scenario_matrix.groupby('Cluster').mean()  


cluster_sizes = scenario_matrix['Cluster'].value_counts().sort_index()


cluster_means['Count'] = cluster_sizes.values


cluster_means.to_excel('cluster_averages.xlsx')


print(cluster_means)